In [ ]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="1" # NVIDIA GeForce RTX 3090
os.environ["CUDA_VISIBLE_DEVICES"]="3" # NVIDIA GeForce RTX 2080
 
import tensorflow.compat.v2 as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
gpus = tf.config.experimental.list_physical_devices('GPU')
 
print(gpus)

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.layers as layers
import zipfile
import os
import random
import datetime
import h5py
import cv2
import sklearn.metrics 
from functions import *

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
# Set name of files/directories used to save logs and model
name_model = "Final (M0_5(3x3)_2)"
#version = ".1"

filepath_dic = "Results/" + name_model
#filepath_subdic = filepath_dic + "/" + name_model + version

filepath_checkpoint_model = filepath_dic + "/checkpoint.model.keras"
log_dir = filepath_dic + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
# Make directory and subdirectory to save logs and model
os.mkdir(filepath_dic)
#os.mkdir(filepath_subdic)

In [ ]:
# Clear any logs from previous runs
#!rm -rf ./logs/

# Data generator

In [ ]:
# Path of hdf5 files
fileHDFTrain = 'Data/train.hdf5'
fileHDFVal = 'Data/val.hdf5'

## Parameters/Settings

In [ ]:
batch_size = 2

augmentations = {
    'random_vertical_flip': None,
    'random_horizontal_flip': None,
    'rotation': 45,
    'translation': 500,
    'brightness': 0.2
}

In [ ]:
train_gen = create_hdf5_generator(
    fileHDFTrain,
    batch_size,
    augmentations = augmentations,
    keys = ['features', 'targets']
)

val_gen = create_hdf5_generator(
    fileHDFVal,
    batch_size,
    augmentations = {},
    keys = ['features', 'targets']
)

## Examples

In [ ]:
features, targets = next(train_gen)

for i in range(batch_size):
    plt.figure()
    plt.subplot(1,2,1)
    plt.imshow(features[i], cmap="gray")
    plt.axis("off")
    plt.title('Image')
    plt.subplot(1,2,2)
    plt.imshow(targets[i], cmap="inferno")
    plt.title('Target')
    plt.axis("off")

In [ ]:
features, targets = next(val_gen)

for i in range(batch_size):
    plt.figure()
    plt.subplot(1,2,1)
    plt.imshow(features[i], cmap="gray")
    plt.title('Image')
    plt.axis("off")
    plt.subplot(1,2,2)
    plt.imshow(targets[i], cmap="inferno")
    plt.title('Target')
    plt.axis("off")

# U-Net


In [ ]:
train = h5py.File(fileHDFTrain, 'r')
val = h5py.File(fileHDFVal, 'r')

## Parameters

In [ ]:
# Determine height and width of the model's inputs
h_img = train["features"].shape[1]
w_img = train["features"].shape[2]

c_feat = 1
num_classes = 1

In [ ]:
# Calculate number of steps per epoch for train and validation
steps_per_epoch = len(train["features"])//batch_size
validation_steps = len(val["features"])//batch_size

print('steps_per_epoch:', steps_per_epoch)
print('validation_steps:', validation_steps)

## Model

In [ ]:
model = initiate_model0(
    h_img=2048, w_img=2688,
    c_feat=1, num_classes=1,
    n_contractions=6,
    dropout_rate=0,
    size_kernel=(3,3))
model.summary()

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.compile(optimizer='adam', loss='binary_crossentropy')

callbacks = [
        tf.keras.callbacks.EarlyStopping(patience= 10, monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir= log_dir),
        tf.keras.callbacks.ModelCheckpoint(filepath=filepath_checkpoint_model ,monitor="val_loss", verbose=True, save_best_only=True)
]

## Training

In [ ]:
history = model.fit(
    train_gen,
    validation_data=(val_gen),
    batch_size=batch_size,
    epochs=50,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=callbacks
)

### Evaluation (loss)

In [ ]:
history_dict = history.history

loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]

#precision = history_dict["precision"]
#val_precicion = history_dict["val_precision"]
#recall = history_dict["recall"]
#val_recall = history_dict["val_recall"]

epochs = range(1, len(loss_values) + 1)

In [ ]:
plt.figure()
plt.plot(epochs, loss_values, "bo", label="Training loss")
plt.plot(epochs, val_loss_values, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.ylim(bottom=0)
plt.legend()
plt.show()

In [ ]:
#plt.plot(epochs, prec, "bo", label="Training precision")
#plt.plot(epochs, val_prec, "b", label="Validation precision")
#plt.title("Training and validation precision")
#plt.xlabel("Epochs")
#plt.ylabel("precision")
#plt.ylim((0,1))
#plt.legend()
#plt.show()

In [ ]:
#plt.plot(epochs, recall, "bo", label="Training recall")
#plt.plot(epochs, val_recall, "b", label="Validation recall")
#plt.title("Training and validation recall")
#plt.xlabel("Epochs")
#plt.ylabel("Recall")
#plt.ylim((0,1))
#plt.legend()
#plt.show()

In [ ]:
%tensorboard --logdir "logs/"